In [ ]:
from pathlib import Path
import numpy as np
import h5py
import pandas
import glob


path2signal = Path.home()/'group'/'project'/'results'/'beamforming'/'time_dependent'/'deep_learning'

path2combineddata = Path.home()/'group'/'project'/'results'/'beamforming'/'time_dependent'/'beamform_signals'

testname = '220605_dl_grid_85to88_5mm_gradb_sum_0_0.npz'

combined_data_name = '220606_dl_grid_85to88_5mm.h5'

In [ ]:
test_file = np.load(path2signal/testname)

In [ ]:
list(test_file.keys())

In [ ]:
list_of_summed_signals = sorted(path2signal.glob('220605_dl_grid_*'))

In [ ]:
print(len(list_of_summed_signals))

In [ ]:
total_N_signal = 0
signal_length = 0

for ipath in list_of_summed_signals:
    total_N_signal += np.load(ipath)['signals'].shape[0]
    
    #print(np.load(ipath)['signals'].shape[0])
    signal_length = np.load(ipath)['signals'].shape[1]
    
print(total_N_signal, signal_length)

In [ ]:
try:
    h5file = h5py.File(path2combineddata/combined_data_name, 'w')

    h5file.create_dataset('x', (total_N_signal, signal_length), dtype='c16')
    h5file.create_dataset('radius', (total_N_signal,), dtype='f8')
    h5file.create_dataset('pitch', (total_N_signal,), dtype='f8')
    h5file.create_dataset('energy', (total_N_signal,), dtype='f8')
    h5file.create_dataset('freq', (total_N_signal,), dtype='f8')

    counter = 0
    
    print(h5file['radius'][10])

    for ipath in list_of_summed_signals:

        isignals = np.load(ipath)['signals'][:]
        #h5file['x'][counter:counter+isignals.shape[0], :] = isignals
        for n in range(isignals.shape[0]):
            for i, key in enumerate(['signals', 'radius', 'pitch', 'energy', 'freq']):
                #print(key)
                if i == 0:
                    h5file['x'][counter] = np.load(ipath)[key][n]
                elif i == 1:
                    h5file[key][counter] = np.load(ipath)[key]
                else:
                    h5file[key][counter] = np.load(ipath)[key][n]
            counter += 1
        
finally:
    h5file.close()

In [ ]:
h5file = h5py.File(path2combineddata/combined_data_name, 'r')

num_0 = 0
for i in range(h5file['x'].shape[0]):
    
    if abs(h5file['x'][i, :]).mean() < 5e-8:
        num_0 += 1
        
        
    if i % 5000 == 4999:
        
        print(i+1)
    

In [ ]:
print(num_0, np.unique(h5file['pitch']), np.unique(h5file['energy']), np.unique(h5file['radius']))

In [ ]:
print(np.unique(h5file['pitch']).size, np.unique(h5file['energy']).size)


